In [53]:
install.packages("caret")
install.packages("dplyr")
install.packages("readr")
library(dplyr)
library(caret)

Installing package into 'C:/Users/USER/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)

also installing the dependencies 'listenv', 'parallelly', 'future', 'globals', 'shape', 'future.apply', 'numDeriv', 'progressr', 'SQUAREM', 'diagram', 'lava', 'prodlim', 'proxy', 'iterators', 'Rcpp', 'clock', 'gower', 'hardhat', 'ipred', 'timeDate', 'e1071', 'foreach', 'ModelMetrics', 'plyr', 'pROC', 'recipes', 'reshape2'




package 'listenv' successfully unpacked and MD5 sums checked
package 'parallelly' successfully unpacked and MD5 sums checked
package 'future' successfully unpacked and MD5 sums checked
package 'globals' successfully unpacked and MD5 sums checked
package 'shape' successfully unpacked and MD5 sums checked
package 'future.apply' successfully unpacked and MD5 sums checked
package 'numDeriv' successfully unpacked and MD5 sums checked
package 'progressr' successfully unpacked and MD5 sums checked
package 'SQUAREM' successfully unpacked and MD5 sums checked
package 'diagram' successfully unpacked and MD5 sums checked
package 'lava' successfully unpacked and MD5 sums checked
package 'prodlim' successfully unpacked and MD5 sums checked
package 'proxy' successfully unpacked and MD5 sums checked
package 'iterators' successfully unpacked and MD5 sums checked
package 'Rcpp' successfully unpacked and MD5 sums checked
package 'clock' successfully unpacked and MD5 sums checked
package 'gower' successf

Warning message:
"package 'dplyr' is in use and will not be installed"
Installing package into 'C:/Users/USER/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)



package 'readr' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\USER\AppData\Local\Temp\RtmpWGg1yJ\downloaded_packages


Cargando paquete requerido: ggplot2

Cargando paquete requerido: lattice



In [4]:
##As in Group Assignment 1 2024 - 2 #1044 , generate the extra-flexible model. This means that it contains all two-way interactions between the experience polynomials and the indicator variables

In [87]:
install.packages("fastDummies")

Installing package into 'C:/Users/USER/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)



package 'fastDummies' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\USER\AppData\Local\Temp\RtmpWGg1yJ\downloaded_packages


In [89]:
# Cargar las librerías necesarias
library(data.table)
library(fastDummies)  # Usar fastDummies para crear variables dummy

# Cargar los datos desde la URL
data_url <- "https://raw.githubusercontent.com/d2cml-ai/CausalAI-Course/main/data/wage2015_subsample_inference.csv"
df <- fread(data_url)

# Revisar la estructura de los datos
str(df)

# Asegurarse de que 'occ2' e 'ind2' sean factores
df$occ2 <- as.factor(df$occ2)
df$ind2 <- as.factor(df$ind2)

# Verificar valores únicos
cat("Valores únicos de occ2:\n", unique(df$occ2), "\n")
cat("Valores únicos de ind2:\n", unique(df$ind2), "\n")

# Crear variables dummy usando fastDummies
df_with_dummies <- dummy_cols(df, select_columns = c("occ2", "ind2"), remove_first_dummy = TRUE)

# Verificar cuántas columnas se han agregado
cat("Número de columnas después de agregar variables dummy:", ncol(df_with_dummies), "\n")

# Definir las variables del modelo
extra_flexible_model_vars <- c('exp1', 'exp2', 'exp3', 'exp4', 'hsg', 'scl', 'clg', 'ad', 
                                'so', 'we', 'ne')

# Filtrar las columnas de variables dummy
dummy_vars <- names(df_with_dummies)[grepl("occ2_|ind2_", names(df_with_dummies))]

# Crear un vector con las variables del modelo
all_vars <- c(extra_flexible_model_vars, dummy_vars)

# Generar interacciones de dos vías
interactions <- NULL
for (i in 1:(length(all_vars) - 1)) {
  for (j in (i + 1):length(all_vars)) {
    interaction_var <- df_with_dummies[[all_vars[i]]] * df_with_dummies[[all_vars[j]]]
    interactions <- cbind(interactions, interaction_var)
  }
}

# Combinar las variables originales y las interacciones
extra_flexible_model_array <- cbind(df_with_dummies[, all_vars, with = FALSE], interactions)

# Revisar la forma del array
dim(extra_flexible_model_array)

# Verificar cuántas variables dummy se generaron
cat("Número de variables dummy:", length(dummy_vars), "\n")
cat("Número total de variables:", length(all_vars), "\n")
cat("Número de interacciones:", ncol(interactions), "\n")


Classes 'data.table' and 'data.frame':	5150 obs. of  21 variables:
 $ rownames: int  10 12 15 18 19 30 43 44 47 71 ...
 $ wage    : num  9.62 48.08 11.06 13.94 28.85 ...
 $ lwage   : num  2.26 3.87 2.4 2.63 3.36 ...
 $ sex     : num  1 0 0 1 1 1 1 0 1 1 ...
 $ shs     : num  0 0 0 0 0 0 0 0 0 0 ...
 $ hsg     : num  0 0 1 0 0 0 1 1 1 0 ...
 $ scl     : num  0 0 0 0 0 0 0 0 0 0 ...
 $ clg     : num  1 1 0 0 1 1 0 0 0 1 ...
 $ ad      : num  0 0 0 1 0 0 0 0 0 0 ...
 $ mw      : num  0 0 0 0 0 0 0 0 0 0 ...
 $ so      : num  0 0 0 0 0 0 0 0 0 0 ...
 $ we      : num  0 0 0 0 0 0 0 0 0 0 ...
 $ ne      : num  1 1 1 1 1 1 1 1 1 1 ...
 $ exp1    : num  7 31 18 25 22 1 42 37 31 4 ...
 $ exp2    : num  0.49 9.61 3.24 6.25 4.84 ...
 $ exp3    : num  0.343 29.791 5.832 15.625 10.648 ...
 $ exp4    : num  0.24 92.35 10.5 39.06 23.43 ...
 $ occ     : num  3600 3050 6260 420 2015 ...
 $ occ2    : int  11 10 19 1 6 5 17 17 13 10 ...
 $ ind     : num  8370 5070 770 6990 9470 7460 7280 5680 8590 8190 .

[1] 5150 1378

Número de variables dummy: 41 
Número total de variables: 52 
Número de interacciones: 1326 


In [ ]:
##2.1. Generate the array for the outcome variable Y and normalize it

In [90]:
log_w <- as.matrix(df$lwage)

In [92]:
normalize_array <- function(arr) {
  min_vals <- apply(arr, 2, min)
  max_vals <- apply(arr, 2, max)
  
  # Calcular el rango y evitar la división por cero
  range_vals <- max_vals - min_vals
  range_vals[range_vals == 0] <- 1  # Previene la división por cero
  
  norm_arr <- (arr - min_vals) / range_vals
  return(norm_arr)
}

In [93]:
norm_log_w <- normalize_array(log_w)

In [94]:
print(norm_log_w)
x_data <- normalize_array(extra_flexible_model_array)
print(x_data)

               [,1]
   [1,] 0.224104735
   [2,] 0.535722332
   [3,] 0.251165289
   [4,] 0.296046462
   [5,] 0.436816837
   [6,] 0.262606019
   [7,] 0.358311129
   [8,] 0.358311129
   [9,] 0.266999506
  [10,] 0.358311129
  [11,] 0.337911342
  [12,] 0.267309544
  [13,] 0.267309544
  [14,] 0.289252119
  [15,] 0.326844393
  [16,] 0.430252856
  [17,] 0.380806113
  [18,] 0.203704948
  [19,] 0.358311129
  [20,] 0.285259836
  [21,] 0.244504522
  [22,] 0.259405634
  [23,] 0.343216301
  [24,] 0.358311129
  [25,] 0.326348570
  [26,] 0.358311129
  [27,] 0.302610443
  [28,] 0.266999506
  [29,] 0.332456928
  [30,] 0.362686884
  [31,] 0.510971376
  [32,] 0.296046462
  [33,] 0.233551433
  [34,] 0.297581098
  [35,] 0.266447097
  [36,] 0.259405634
  [37,] 0.401515938
  [38,] 0.325966301
  [39,] 0.259405634
  [40,] 0.505875847
  [41,] 0.381116151
  [42,] 0.253951220
  [43,] 0.155045725
  [44,] 0.233551433
  [45,] 0.267309544
  [46,] 0.297156029
  [47,] 0.436816837
  [48,] 0.168404049
  [49,] 0.263397918


In [ ]:
##Split between training and testing samples. The testing sample should be 10% of the total.

In [95]:
install.packages("caTools")
library(caTools)

Installing package into 'C:/Users/USER/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)

also installing the dependency 'bitops'




package 'bitops' successfully unpacked and MD5 sums checked
package 'caTools' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\USER\AppData\Local\Temp\RtmpWGg1yJ\downloaded_packages


In [96]:
ef_model <- x_data
y <- norm_log_w                                                                                                                                            # Definir el tamaño de la muestra de prueba (10% del total)
set.seed(123)  # Para reproducibilidad
split <- sample.split(y, SplitRatio = 0.1)

# Dividir las muestras en entrenamiento y prueba
ef_train <- ef_model[split, ]
ef_test <- ef_model[!split, ]
y_ef_train <- y[split, ]
y_ef_test <- y[!split, ]

# Mostrar las dimensiones para verificar la correcta división
cat("Dimensiones de entrenamiento (ef_train):", dim(ef_train), "\n")
cat("Dimensiones de prueba (ef_test):", dim(ef_test), "\n")
cat("Dimensiones de entrenamiento (y_ef_train):", dim(y_ef_train), "\n")
cat("Dimensiones de prueba (y_ef_test):", dim(y_ef_test), "\n")

Dimensiones de entrenamiento (ef_train): 1383 1378 
Dimensiones de prueba (ef_test): 3767 1378 
Dimensiones de entrenamiento (y_ef_train): 
Dimensiones de prueba (y_ef_test): 
